[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github//ragatti/demucs/blob/master/demucs_colab.ipynb) 

[![View source on GitHub!](https://badgen.net/badge/icon/View%20on%20Github?icon=github&label)](https://github.com/ragatti/demucs/blob/master/demucs_colab.ipynb)

# Import and functions

In [ ]:
#@title
!git clone https://github.com/ragatti/demucs
%cd /content/demucs
import os
import IPython.display as ipd
from pathlib import Path
from ipywidgets import VBox, HBox, widgets
import functools
from google.colab import files
cwd = os.getcwd()

In [ ]:
#@title
def on_click_play(change, file=None):
  out.clear_output()
  with out:
    if file:
      out_file = Path.joinpath(out_path, "{}.wav".format(file))
      try:
        display(ipd.Audio(out_file.read_bytes(), autoplay=True))
      except FileNotFoundError:
        print("The file '{}' does not exists.\n".format(file),
              "You should separate the uploaded audio by\n",
              "pressing the demucs button", sep='')
    else:
      try:
        display(ipd.Audio(btn_upload.data[-1], autoplay=True))
      except IndexError:
        print("You should upload a mp3 file")        

def on_click_save(change, file='original'):
  files.download(str(Path.joinpath(out_path,
                     "{}.wav".format(file))))

def on_click_demucs(change):
  out.clear_output()
  in_path.mkdir(exist_ok=True)
  try:
    in_file.write_bytes(btn_upload.data[-1])
    print('Please wait ...')
    !python3 -m demucs.separate --dl -n demucs -d cpu \
      "./input/original.mp3"
  except IndexError:
    print("You should upload a file")

# Widget

In [4]:
#@title
in_path = Path(cwd, 'input')
in_file = Path.joinpath(in_path, "original.mp3")
out_path = Path(cwd, 'separated', 'demucs', 'original')
btn_upload = widgets.FileUpload(multiple=False)
songs = ['original', 'bass', 'drums', 'vocals', 'other']
btn_demucs = widgets.Button(description='Demucs',
                            button_style='warning')
play_btns = [widgets.Button(description='Play {}'.format(song)) 
             for song in songs]
save_btns = [widgets.Button(description='Save'.format(song),
                            button_style='info') 
             for song in songs]
out = widgets.Output()
btn_demucs.on_click(on_click_demucs)
for i, song in enumerate(songs):
  play_btns[i].on_click(functools.partial(on_click_play,
                                          file=song))
  save_btns[i].on_click(functools.partial(on_click_save,
                                          file=song))
play_btns[0].on_click(functools.partial(on_click_play,
                                        file=None))

left_box = VBox([play_btns[i] for i in range(1,len(song))])
right_box = VBox([save_btns[i] for i in range(1,len(song))])
VBox([widgets.Label('Upload audio file'), btn_upload,
      HBox([play_btns[0], btn_demucs]),
      HBox([left_box, right_box]), out])